##### 네이버 카페 크롤러
- 필요 라이브러리 import

In [44]:
from webdriver_manager.chrome import ChromeDriverManager

from collections import Counter
import pickle
from tqdm import tqdm

import matplotlib.pyplot as plt
import seaborn as sns

from matplotlib import font_manager, rc  
font_name = font_manager.FontProperties(fname='C:/Windows/Fonts/malgun.ttf').get_name()
rc('font', family=font_name)

In [45]:
#selenium 셋팅
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By

In [46]:
import time
import csv
import pandas as pd
import numpy as np
import random
from bs4 import BeautifulSoup as bs

##### 네이버 로그인
- 자동화된 크롬창을 실행시켜서 네이버로 로그인 페이지 접속
1. 크롬드라이버 제어 (크롬창 켜기)
2. 네이버 로그인(아이디입력 -> 비밀번호 입력 -> 로그인 버튼 클릭)

In [47]:
#크롬드라이버 제어
chrome_options = Options()
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dex-shm-usage')

#현재 컴푸터 크롬 드라이버 위치
#본인의 크롬 드라이버 위치 입력
chrome_path = 'chromedriver.exe'

#자동화된 크롬 창 실행
driver = webdriver.Chrome(executable_path=chrome_path, chrome_options=chrome_options)

C:\Users\user\AppData\Local\Temp\ipykernel_17204\3214283744.py:11: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(executable_path=chrome_path, chrome_options=chrome_options)


In [48]:
# 네이버 로그인 페이지 접속
login_url = 'https://nid.naver.com/nidlogin.login?mode=form&url=https%3A%2F%2Fwww.naver.com'
driver.get(login_url)
driver.implicitly_wait(10)

#본인의 아이디, 비밀번호를 각 변수에 저장
my_id='scv0206'
my_pw='DlDpWl026!@'

# 아이디, 비밀번호 입력 (네이버에 로그인 할 경우 'send_keys()' 함수가 아니라 'execute_script()' 사용)
#네이버에서 send_key()를 쓰면 로그인에서 자동입력방지기능이 나타남.
driver.execute_script("document.getElementsByName('id')[0].value = \'" + my_id + "\'")
driver.execute_script("document.getElementsByName('pw')[0].value = \'" + my_pw + "\'")
time.sleep(1)

#로그인 버튼 클릭
driver.find_element('id', 'log.login').click()
time.sleep(1)

In [49]:
#카페 접속: https://cafe.naver.com/workee
time.sleep(2)
#타겟 카페가 바뀔 경우 url을 바꿔주세요!
moms_url='https://cafe.naver.com/workee'
driver.get(moms_url)
time.sleep(1)
#원하는 게시판 클릭
#게시판의 선택자를 넣어주기
driver.find_element('xpath', '//*[@id="menuLink174"]').click()
time.sleep(1)

#프레임 전환-iframe-전체선택되는 것 - 검색 안되면 ctrl+f
driver.switch_to.frame('cafe_main')
time.sleep(1)

##### 게시물 크롤링
1. 첫번째 게시물 클릭 - XPath활용
2. 크롤링할 개수 지정
3. 크롤링

In [51]:
# 제목, 본문, 댓글, 시간, 빈 list 생성
#제목
titles = []
#본문
contents=[]
#리뷰
#reviews = []

#댓글
comments=[]
#시간
date=[]

count_view=0

#1.첫번째 게시물 클릭(선택자 입력: XPath)
driver.find_element('xpath', '//*[@id="main-area"]/div[4]/table/tbody/tr[1]/td[1]/div[2]/div/a[1]').click()

#2.크롤링(for문으로 지정한 게시물 개수까지 돌림)
for i in range(2):
    for i in range(30): #한번에 몇개의 게시물을 크롤링할 것인가
        driver.implicitly_wait(30)

        #제목 수집
        time.sleep(random.uniform(0.5, 1))
        title = driver.find_element('xpath', '//*[@id="app"]/div/div/div[2]/div[1]/div[1]/div/h3').click()
        titles.append(title)
        
        #본문 수집(3가지 경우 존재-글전체, 이미지, 한줄한줄 인지)
        try: 
            content = driver.find_element(By.CSS_SELECTOR, 'div.se-module.se-module-text')
        except:
            try:
                content = driver.find_element(By.CSS_SELECTOR, 'div.ContentRenderer')
            except:
                #본문이 사진으로만 구성된 이유
                content=[]
                contents.append(content)


        #댓글 수집(댓글이 없는 경우, 댓글이 있는 경우 2가지 존재)
        #현재 페이지 HTML 정보 가지고 오기
        soup = bs(driver.page_source, 'lxml')
        #                                        text_comment
        iscomment = soup.find_all('span', class_='comment_text_view')
        if len(iscomment) == 0: #댓글이 없는 경우
            box=[]
        else: #댓글이 있는 경우
            WebDriverWait(driver, 15).until(EC.presence_of_all_elements_located((By.CLASS_NAME, 'text_comment')))
            #웹드라이버를 통해 브라우저에서 최대 15초 기다려주고, text_comment(댓글 선택)

            soup = bs(driver.page_source, 'lxml')
            iscomment = soup.find_all('spand', class_='comment_text_view')
            box=[]
            for i in iscomment:
                box.append([i.get_text()])
        comments.append(box)


        #날짜 수집
        times = driver.find_element('xpath', '//*[@id="app"]/div/div/div[2]/div[1]/div[2]/div/div[2]/span[1]')
        date.append(times)
        #count_view 크롤링 진행 상황(50개마다 출력)
        count_view = count_view + 1
        if count_view % 50 ==0: #50으로 나누었을 때 0이 되면 게시물 크롤링 완료라고 뜨도록 설정
            print('{}개 게시물 크롤링 완료'.format(count_view))
        else:
            pass

        #다음글 클릭(다음글 버튼 클릭해서 연속적으로 크롤링, 첫번째글->두번째글->세번째글...)
        try:
            driver.find_element(By.CSS_SELECTOR, '#app > div >')
            driver.implicitly_wait(28)
        except:
            current_url = driver.current_url
            driver.close()

            #크롬창 다시 열기
            driver = webdriver.Chrome(executable_path=chrome_path, chrome_options=chrome_options)

            #재로그인
            #네이버 로그인 화면 이동
            # 네이버 로그인 페이지 접속
            login_url = 'https://nid.naver.com/nidlogin.login?mode=form&url=https%3A%2F%2Fwww.naver.com'
            driver.get(login_url)
            driver.implicitly_wait(10)

            #본인의 아이디, 비밀번호를 각 변수에 저장
            my_id='scv0206'
            my_pw='DlDpWl026!@'

            # 아이디, 비밀번호 입력 (네이버에 로그인 할 경우 'send_keys()' 함수가 아니라 'execute_script()' 사용)
            #네이버에서 send_key()를 쓰면 로그인에서 자동입력방지기능이 나타남.
            driver.execute_script("document.getElementsByName('id')[0].value = \'" + my_id + "\'")
            driver.execute_script("document.getElementsByName('pw')[0].value = \'" + my_pw + "\'")
            time.sleep(1)

            #로그인 버튼 클릭
            driver.find_element('id', 'log.login').click()
            time.sleep(1)

            #멈친 부분의 링크 가지고 오기
            driver.get(current_url)
            driver.switch_to_frame('cage_main')
            time.sleep(4)

    # 크롬이 다시 열릴 때 가지고 올 다음 링크 가져오기 #다음글 클릭에 해당되는 href 추출 (url로 이동해야 하기 때문에)
        link = driver.find_element('xpath','//*[@id="app"]/div/div/div[1]/div[2]/a[2]').get_attribute("href")

    #현재 크롬창 닫기
    driver.close()

    #크롬창 다시 열기
    driver = webdriver.Chrome(executable_path=chrome_path, chrome_options=chrome_options)


    #재로그인
    #네이버 로그인 화면 이동
    # 네이버 로그인 페이지 접속
    login_url = 'https://nid.naver.com/nidlogin.login?mode=form&url=https%3A%2F%2Fwww.naver.com'
    driver.get(login_url)
    driver.implicitly_wait(10)

    #본인의 아이디, 비밀번호를 각 변수에 저장
    my_id='scv0206'
    my_pw='DlDpWl026!@'

    # 아이디, 비밀번호 입력 (네이버에 로그인 할 경우 'send_keys()' 함수가 아니라 'execute_script()' 사용)
    #네이버에서 send_key()를 쓰면 로그인에서 자동입력방지기능이 나타남.
    driver.execute_script("document.getElementsByName('id')[0].value = \'" + my_id + "\'")
    driver.execute_script("document.getElementsByName('pw')[0].value = \'" + my_pw + "\'")
    time.sleep(1)

    #로그인 버튼 클릭
    driver.find_element('id', 'log.login').click()
    time.sleep(1)

    #다음 링크부터 가져오기
    driver.get(link)
    driver.switch_to_frame("cafe.main")
        
#크롬 창 닫기
driver.quit()



NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="main-area"]/div[4]/table/tbody/tr[1]/td[1]/div[2]/div/a[1]"}
  (Session info: chrome=107.0.5304.107)


In [ ]:
# 2-2. 게시판에서 크롤링 원하는 영역 지정(ifram) - 범위 외에서 크롤링이 되지 않도록
driver.switch_to.frame('cafe_main'); time.sleep(1)

# 2-3. 크롤링하기(제목, 본문, 댓글, 시간 빈 list 생성)
titles = [] # 제목
reviews = [] # 리뷰
comments = [] # 댓글
dates = [] # 시간

count_view = 0

# 1) 첫번째 게시물 클릭
driver.find_element(By.XPATH, '//*[@id="main-area"]/div[4]/table/tbody/tr[1]/td[1]/div[2]/div').click()

# 2. 크롤링 for문 돌리기
for a in range(2): #새로 창 열기
    for b in range(10): # 한 번에 크롤링할 게시물 수
        driver.implicitly_wait(5)
        # 제목수집
        time.sleep(random.uniform(0.5, 1))
        title = driver.find_element(By.XPATH, '//*[@id="app"]/div/div/div[2]/div[1]/div[1]/div/h3').text
        titles.append(title)
        # 본문 수집(3가지 종류 - 사진이 있는 경우, 수정이된 경우)
        try:
            content = driver.find_elements_by_css_selector('div.se-module.se-module-text>p.se-text-paragraph.se-text-paragraph-align- ')
            for a in content:
                contents = ""
                contents += a.text
                reviews.append(contents)
        except:
            try:
                content = driver.find_element(By.CSS_SELECTOR, 'div.ContentRenderer')
            except:
                content = []
                reviews.append(content)
        
        #댓글 수집(댓글이 없는 경우, 댓글이 있는 경우)
        soup = bs(driver.page_source, 'lxml')
        iscomment = soup.find_all(By.CSS_SELECTOR,'sapn.text_comment')
        if len(iscomment)==0: # 댓글이 없는 경우
            box = []
        else: # 댓글이 있는 경우
            # text_comment가 다 나올때까지 15동안 기다리라는 코드
            WebDriverWait(diver, 5).until(
                                EC.presence_of_all_elements_located(
                                    By.CSS_SELECTOR,'sapn.text_comment'))
            soup = bd(driver.page_source, 'lxml')
            # text_comment를 찾아서 리스트로 만들기 - html코드로 되어 있음
            iscomment = soup.find_all(By.CSS_SELECTOR,'sapn.text_comment')
            box = []
            for c in iscomment:
                box.append([c.get_text()])
        comments.append(box)
            
        # 날짜 수집
        date = driver.find_element(By.CSS_SELECTOR, 'div.article_info>span.date').text
        dates.append(d)
        
        # 크롤링 진행 상황(50개마다 출력)
        count_view = count_view + 1
        if count_view % 50 == 0:
            print(f'{count_view}개 게시물 크롤링 완료')
        
        # 다음 글 클릭
        driver.find_element(By.CSS_SELECTOR, 'a.BaseButton.btn_next.BaseButton--skinGray.size_default').click()

print(titles, date, comments, reviews)


In [52]:
# 2-2. 게시판에서 크롤링 원하는 영역 지정(ifram) - 범위 외에서 크롤링이 되지 않도록
driver.switch_to.frame('cafe_main'); time.sleep(1)

# 2-3. 크롤링하기(제목, 본문, 댓글, 시간 빈 list 생성)
titles = [] # 제목
reviews = [] # 리뷰
comments = [] # 댓글
date = [] # 시간

count_view = 0

# 1) 첫번째 게시물 클릭
driver.find_element(By.XPATH, '//*[@id="main-area"]/div[4]/table/tbody/tr[1]/td[1]/div[2]/div').click()

# 2. 크롤링 for문 돌리기
for a in range(2): #새로 창 열기
    for b in range(10): # 한 번에 크롤링할 게시물 수
        driver.implicitly_wait(5)
        # 제목수집
        time.sleep(random.uniform(0.5, 1))
        title = driver.find_element(By.XPATH, '//*[@id="app"]/div/div/div[2]/div[1]/div[1]/div/h3').text
        titles.append(title)
        # 본문 수집(3가지 종류 - 사진이 있는 경우, 수정이된 경우)
        try:
            content = driver.find_element(By.CSS_SELECTOR, 'div.se-module.se-module-image').text
            reviews.append(content)
        except:
            try:
                content = driver.find_element(By.CSS_SELECTOR, 'div.ContentRenderer')
            except:
                content = []
                reviews.append(content)
        
        #댓글 수집(댓글이 없는 경우, 댓글이 있는 경우)
        soup = bs(driver.page_source, 'lxml')
        iscomment = soup.find_all('sapn', class_='text_comment')
        if len(iscomment)==0: # 댓글이 없는 경우
            box = []
        else: # 댓글이 있는 경우
            # text_comment가 다 나올때까지 15동안 기다리라는 코드
            WebDriverWait(diver, 5).until(
                                EC.presence_of_all_elements_located(
                                    By.CLASS_NAME, 'text_comment'))
            soup = bd(driver.page_source, 'lxml')
            # text_comment를 찾아서 리스트로 만들기 - html코드로 되어 있음
            iscomment = soup.find_all('sapn', class_='text_comment')
            box = []
            for c in iscomment:
                box.append([c.get_text()])
        comments.append(box)
            
        # 날짜 수집
        d = driver.find_element(By.CSS_SELECTOR, 'div.article_info>span.date').text
        date.append(d)
        
        # 크롤링 진행 상황(50개마다 출력)
        count_view = count_view + 1
        if count_view % 50 == 0:
            print(f'{count_view}개 게시물 크롤링 완료')
        
        # 다음 글 클릭
        driver.find_element(By.CSS_SELECTOR, 'a.BaseButton.btn_next.BaseButton--skinGray.size_default').click()

print(titles, date, comments, reviews)


NoSuchFrameException: Message: cafe_main
